<h1><center>Laboratorio 7: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Matías Rojas y Mauricio Araneda
- Auxiliar: Ignacio Meza D.
- Ayudante: Rodrigo Guerra

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Felipe Aguilar
- Nombre de alumno 2: Joaquín Pérez O.


### **Link de repositorio de GitHub:** `https://github.com/JoaquinPerez17/Lab-MDS7202`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- Fecha de entrega: TBD
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.3 MB 28.0 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0


https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [107]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = '/content/drive/MyDrive/Laboratorios MDS/Laboratorio_7/'
except: 
    print('Ignorando conexión drive-colab')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [321]:
df_players = pd.read_csv(path + 'stats_players.csv')
df_players

Name          Nationality National_Position Club_Position  \
0      Cristiano Ronaldo             Portugal                LS            LW   
1           Lionel Messi            Argentina                RW            RW   
2                 Neymar               Brazil                LW            LW   
3            Luis Suárez              Uruguay                LS            ST   
4           Manuel Neuer              Germany                GK            GK   
...                  ...                  ...               ...           ...   
17583        Adam Dunbar  Republic of Ireland               NaN           Sub   
17584       Dylan McGoey  Republic of Ireland               NaN           Sub   
17585    Tommy Ouldridge              England               NaN           Res   
17586         Mark Foden             Scotland               NaN           Sub   
17587   Barry Richardson              England               NaN           Sub   

       Height  Weight Preffered_Foot  Age        Work_Rate  Weak_foot  ...  \
0         185      80          Right   32       High / Low          4  ...   
1         170      72           Left   29  Medium / Medium          4  ...   
2         174      68          Right   25    High / Medium          5  ...   
3         182      85          Right   30    High / Medium          4  ...   
4         193      92          Right   31  Medium / Medium          4  ...   
...       ...     ...            ...  ...              ...        ...  ...   
17583     183      82          Right   19  Medium / Medium          1  ...   
17584     185      80          Right   19  Medium / Medium          2  ...   
17585     173      61          Right   18    High / Medium          2  ...   
17586     180      80          Right   21  Medium / Medium          3  ...   
17587     185      77          Right   47  Medium / Medium          2  ...   

       Agility  Jumping  Heading  Shot_Power  Finishing  Long_Shots  Curve  \
0           90       95       85          92         93          90     81   
1           90       68       71          85         95          88     89   
2           96       61       62          78         89          77     79   
3           86       69       77          87         94          86     86   
4           52       78       25          25         13          16     14   
...        ...      ...      ...         ...        ...         ...    ...   
17583       27       56       14          16         13          13     11   
17584       28       53       12          17         12          11     12   
17585       54       61       41          44         28          42     35   
17586       34       48       15          23         14          12     13   
17587       38       51       12          13         11          16     12   

       Freekick_Accuracy  Penalties  Volleys  
0                     76         85       88  
1                     90         74       85  
2                     84         81       83  
3                     84         85       88  
4                     11         47       11  
...                  ...        ...      ...  
17583                 13         15       12  
17584                 13         16       12  
17585                 36         42       37  
17586                 12         24       12  
17587                 11         22       12  

[17588 rows x 39 columns]

## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [322]:
### Código aquí ###
df_players['Labels'] = df_players.loc[:, 'National_Position'].apply(lambda x: 0 if pd.isna(x) else 1)
df_players['Labels'].value_counts()

0    16513
1     1075
Name: Labels, dtype: int64

Se observa que existe una amplia mayoría de jugadores no seleccionados.

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [ ] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [ ] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [ ] Crear 3 ``pipelines`` con diferentes clasificadores.
- [ ] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [ ] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta:**

En cuanto a la elección de las transformaciones en las columnas, se siguió la siguiente idea; Si los datos tienen un comportamiento normal o relativamente normal, se usa la Estandirazación, si los datos no tienen un comportamiento normal, se usa el Escalamiento mínimo-máximo. Como en ninguna columna hay una presencia significativa de outliers, se descarta el uso de una Transformación robusta.

In [232]:
cat_cols = ['Nationality', 'Club_Position', 'Preffered_Foot']

ord_cols = ['Weak_foot', 'Skill_Moves', 'Work_Rate']

ord_cat = [['1', '2', '3', '4', '5'], ['1', '2', '3', '4', '5'], 
           ['Low / Low', 'High / Low', 'Low / High', 'Low / Medium','Medium / Low', 'Medium / Medium', 'Medium / High', 'High / Medium', 'High / High']]

normal_cols = ['Height', 'Weight', 'Aggression', 'Reactions', 'Vision', 'Composure', 'Speed', 'Strength', 'Balance', 'Agility', 'Jumping', 'Penalties', 'Volleys']

no_normal_cols = ['Age', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle', 'Standing_Tackle', 'Interceptions', 'Crossing', 'Short_Pass', 
                                 'Long_Pass' ,'Acceleration', 'Stamina', 'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve', 'Freekick_Accuracy']


In [233]:
from sklearn.preprocessing import OrdinalEncoder

preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(sparse=False, handle_unknown='ignore'),  cat_cols),
        ('OrdinalEncoder', OrdinalEncoder(categories=ord_cat), ord_cols),
        ('StandardScaler', StandardScaler(), normal_cols),
        ('MinMaxScaler', MinMaxScaler(), no_normal_cols)])

In [234]:
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

linearSVC_pipe = Pipeline(
    [
        ('preprocesamiento', preprocessing_transformer),
        ('linearsvc', LinearSVC()),
    ]
)

knn_pipe = Pipeline(
    [
        ('preprocesamiento', preprocessing_transformer), 
        ('knn', KNeighborsClassifier())
    ]
)

SVC_pipe = Pipeline(
    [
        ('preprocesamiento', preprocessing_transformer),
        ('linearsvc', SVC(kernel='rbf', random_state=42, gamma='scale')),
    ]
)

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [235]:
X = df_players.drop(columns = ['Name', 'National_Position', 'Labels'], axis=1)
y = df_players['Labels']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

In [95]:
linearSVC = linearSVC_pipe.fit(X_train, y_train)
y_pred = linearSVC.predict(X_test)
print('Reporte de clasificación:')
print(classification_report(y_pred, y_test))

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.99      0.96      0.97      5616
           1       0.36      0.68      0.47       189

    accuracy                           0.95      5805
   macro avg       0.68      0.82      0.72      5805
weighted avg       0.97      0.95      0.96      5805



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [96]:
KNN = knn_pipe.fit(X_train, y_train)
y_pred = KNN.predict(X_test)
print('Reporte de clasificación:')
print(classification_report(y_pred, y_test))

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.99      0.94      0.97      5751
           1       0.06      0.39      0.10        54

    accuracy                           0.94      5805
   macro avg       0.53      0.67      0.53      5805
weighted avg       0.99      0.94      0.96      5805



In [100]:
SVC = SVC_pipe.fit(X_train, y_train)
y_pred = SVC.predict(X_test)
print('Reporte de clasificación:')
print(classification_report(y_pred, y_test))

Reporte de clasificación:
              precision    recall  f1-score   support

           0       1.00      0.94      0.97      5805
           1       0.00      0.00      0.00         0

    accuracy                           0.94      5805
   macro avg       0.50      0.47      0.48      5805
weighted avg       1.00      0.94      0.97      5805



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Respuesta**

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?

Accuracy es la razón entre predicciones correctas y totales pero no distingue las clases de las predicciones, Precision calcula qué porcentaje de entidades encontradas por el modelo está presente en las etiquetas reales, y Recall calcula qué porcentaje de entidades presentes en las etiquetas reales es encontrado por el modelo. 

LinearSVC resultó ser el mejor prediciendo si un jugador SÍ es seleccionado nacional, sin embargo, en los 3 clasificadores se obtiene una precisión mucho menor a la precisión de predecir si NO es seleccionado. LinearSVC es el clasificador que tiene mejor Recall prediciendo se un jugador SÍ es seleccionado donde también su valor es similar al de predecir si NO es seleccionado. De todas formas, la performance de los 3 modelos es bastante baja dado el desbalance presente en las clases.

- ¿Podrían mejorarse los resultados?, ¿Cómo?

Si fuese posible a acceder a más datos, una forma de mejorar los resultados sería aumentar la cantidad de ejemplos de clase "seleccionado" para llegar a tener cantidades similares por clase. Si lo anterior no se puede hacer, se podría realizar un remuestreo de los datos, de tal forma de aumentar la cantidad de ejemplos en la clase minoritaria. Por último, se podría realizar un entrenamiento con un enfoque dirigido hacia los pesos de cada clase para poder mantener las proporciones.

- ¿Influye la cantidad de ejemplos por clase?

Dadas las preguntas anteriores, se puede afirmar que la cantidad de ejemplos por clase sí influye en el desempeño de los clasificadores, dado que todos los clasificadores utilizados obtuvieron un desempeño peor para la clase minoritaria (Ser seleccionado nacional) en comparación a la clase mayoritaria (No ser seleccionado nacional). Con tan pocos ejemplos en la clase minoritaria, no es pobile aprender de manera correcta los patrones que logran diferenciar a esa clase.

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [ ] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [ ] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

 - Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.

In [323]:
dic_replace_label = {
          'ST': "ataque", 
          'CF': "ataque",
          'RW': "central_ataque",
          'CAM': "central_ataque",
          'LW': "central_ataque", 
          'RM': "central",
          'CM': "central",
          'LM': "central",
          'RWB': "central_defensa",
          'CDM': "central_defensa",
          'LWB': "central_defensa",
          'RB': "defensa",
          'CB': "defensa",
          'LB': "defensa",
          'GK': "arquero"
          }

df_players["Position"] = df_players["Club_Position"].map(dic_replace_label).astype('category')

 - Cuente cuántos por clase quedan.

In [324]:
df_players["Position"].value_counts()

defensa            1180
central             907
arquero             632
central_ataque      581
ataque              430
central_defensa     209
Name: Position, dtype: int64

- Entrene el nuevo pipeline y ejecute una evaluación de este.

In [237]:
X_players = df_players.drop(['Name', 'National_Position', 'Club_Position', 'Labels'], axis = 1)
X_players = X_players.dropna(subset = ['Position'])

In [119]:
cat_cols = ['Nationality', 'Preffered_Foot']

ord_cols = ['Weak_foot', 'Skill_Moves', 'Work_Rate']

ord_cat = [['1', '2', '3', '4', '5'], ['1', '2', '3', '4', '5'], 
           ['Low / Low', 'High / Low', 'Low / High', 'Low / Medium','Medium / Low', 'Medium / Medium', 'Medium / High', 'High / Medium', 'High / High']]

normal_cols = ['Height', 'Weight', 'Aggression', 'Reactions', 'Vision', 'Composure', 'Speed', 'Strength', 'Balance', 'Agility', 'Jumping', 'Penalties', 'Volleys']

no_normal_cols = ['Age', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle', 'Standing_Tackle', 'Interceptions', 'Crossing', 'Short_Pass', 
                                 'Long_Pass' ,'Acceleration', 'Stamina', 'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve', 'Freekick_Accuracy']


preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(sparse=False, handle_unknown='ignore'),  cat_cols),
        ('OrdinalEncoder', OrdinalEncoder(categories=ord_cat), ord_cols),
        ('StandardScaler', StandardScaler(), normal_cols),
        ('MinMaxScaler', MinMaxScaler(), no_normal_cols)])


linearSVC_pipe = Pipeline(
    [
        ('preprocesamiento', preprocessing_transformer),
        ('linearsvc', LinearSVC()),
    ]
)

knn_pipe = Pipeline(
    [
        ('preprocesamiento', preprocessing_transformer), 
        ('knn', KNeighborsClassifier())
    ]
)

SVC_pipe = Pipeline(
    [
        ('preprocesamiento', preprocessing_transformer),
        ('linearsvc', SVC(kernel='rbf', random_state=42, gamma='scale')),
    ]
)

In [142]:
X = X_players.drop(['Position'], axis=1)
y = X_players.loc[:,"Position"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0, stratify=y)

In [126]:
linearSVC = linearSVC_pipe.fit(X_train, y_train)
KNN = knn_pipe.fit(X_train, y_train)
SVC = SVC_pipe.fit(X_train, y_train)

y_pred_linear = linearSVC.predict(X_test)
y_pred_KNN = KNN.predict(X_test)
y_pred_SVC = SVC.predict(X_test)
print('Reporte de clasificación Linear SVC:')
print(classification_report(y_pred_linear, y_test))

print('Reporte de clasificación KNN:')
print(classification_report(y_pred_KNN, y_test))

print('Reporte de clasificación SVC:')
print(classification_report(y_pred_SVC, y_test))

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


Reporte de clasificación Linear SVC:
                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       209
         ataque       0.84      0.75      0.79       159
        central       0.65      0.58      0.61       332
 central_ataque       0.31      0.60      0.41        98
central_defensa       0.17      0.55      0.26        22
        defensa       0.97      0.79      0.87       480

       accuracy                           0.75      1300
      macro avg       0.66      0.71      0.66      1300
   weighted avg       0.81      0.75      0.77      1300

Reporte de clasificación KNN:
                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       209
         ataque       0.72      0.66      0.69       155
        central       0.53      0.48      0.50       326
 central_ataque       0.26      0.38      0.31       129
central_defensa       0.14      0.26      0.19        39
        defensa  

**Respuesta**:

Si se compara este problema con el que fue resuelto anteriormente, se puede afirmar que los resultados mejoran significativamente en los 3 clasificadores, aunque se mantiene el hecho de que para las clases minoritarias los resultados son muy malos.

Un punto peculiar a destacar, es que la clase Arquero fue predicha correctamente en todos los clasificadores, lo que podría ser explicado a que la clase arquero fue asignada con valores bajos en relación al resto de las posiciones, debido a que las habilidades de un arquero son bajas en movilidad y destreza con el balón a diferencia de otras como reacción o agarre del balón con las manos. Lo anterior permite que esa clase sea completamente separable de las del resto.

Dado los resultados, el problema a mejorar va enfocado a las 2 clases minoritarias, una solución (aparte de obtener más datos) podría ser sobremuestrear esas dos clases y submuestrear las clases mayoritarias. Además, para mejorar los resultados generales, sería útil encontrar los mejores parámetros para cada modelo utilizando una búsqueda por grilla con cross-validation.





## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [ ] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

In [325]:
### Código aquí ###
df_sueldos = pd.read_csv(path + 'sueldos.csv')
df_players = df_players.drop(columns=['Position'])

In [326]:
df_ply_sueldos = df_sueldos.copy().rename(columns={"Player": "Name"})
df_ply_sueldos = df_ply_sueldos.merge(df_players, on="Name")
df_ply_sueldos = df_ply_sueldos.dropna()
df_ply_sueldos = df_ply_sueldos.drop(columns=["Unnamed: 0"])
df_ply_sueldos['Weekly Salary'] = df_ply_sueldos['Weekly Salary'].astype(int)

In [327]:
X_players_sueldo = df_ply_sueldos.drop(['Age', 'Weight', 'Height', 'Nationality', 'Preffered_Foot', 'Club_Position', 'Labels'], axis = 1)

In [331]:
X = X_players_sueldo.drop(columns=['Weekly Salary'], axis=1)
y = X_players_sueldo['Weekly Salary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [332]:
from sklearn import linear_model

ord_cols = ['Weak_foot', 'Skill_Moves', 'Work_Rate']

ord_cat = [['1', '2', '3', '4', '5'], ['1', '2', '3', '4', '5'], 
           ['Low / Low', 'High / Low', 'Low / High', 'Low / Medium','Medium / Low', 'Medium / Medium', 'Medium / High', 'High / Medium', 'High / High']]

normal_cols = ['Aggression', 'Reactions', 'Vision', 'Composure', 'Speed', 'Strength', 'Balance', 'Agility', 'Jumping', 'Penalties', 'Volleys']

no_normal_cols = ['Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle', 'Standing_Tackle', 'Interceptions', 'Crossing', 'Short_Pass', 
                                 'Long_Pass' ,'Acceleration', 'Stamina', 'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve', 'Freekick_Accuracy']


preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('OrdinalEncoder', OrdinalEncoder(categories=ord_cat), ord_cols),
        ('StandardScaler', StandardScaler(), normal_cols),
        ('MinMaxScaler', MinMaxScaler(), no_normal_cols)])

Lasso = linear_model.Lasso(alpha=0.1)
linear_regression = Pipeline(
    [
        ('preprocesamiento', preprocessing_transformer),
        ('linearmodel', Lasso),
    ]
)


In [333]:
linear_regression.fit(X_train, y_train)
print(linear_regression.score(X_test, y_test))

0.44486658455779216


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.412e+10, tolerance: 6.202e+08
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


**Respuesta**:


La métrica $R^{2}$ o coeficiente de determinación, determina la proporción de variación de la variable objetivo que puede explicarse por el modelo.

Se utilizó el modelo "Lasso" para escoger el mejor algoritmo de predicción. con el fin de mejorar el despempeño de la regresión se filtraron varias columnas que en principio no influyen en la destreza del jugadordado que la destreza esta altamente relacionada con el sueldo del jugador. Las columnas filtradas fueron: "Age", "Height", "Weight", "Nationality", "Preffered_Foot" y "Club_Position".

El modelo es dependiente de la aleatoriedad de la partición debido al efecto que tiene al variar el parámetro "RandomState". A modo de mejorar los resultados, un k-fold cross validation entregaría una mejor idea de la capacidad de la regresión de ajustarse a los datos. También se podría tratar de aumentar el $R^2$ con una mayor cantidad de muestras, producto que la dimensión del dataset unido es menor, no obstante, una mayor cantidad de datos no aseguraría que el $R^2$ sea mayor. 


# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>